In [1]:
import cv2
import os
import shutil
import random
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf


from keras.models import Model
from keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

2023-11-19 18:19:54.504833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# RESCALE

def preprocess(file_path):
    
    byte_img = tf.io.read_file(file_path)
     
    img = tf.io.decode_jpeg(byte_img)
   
    img = tf.image.resize(img, (105,105))
    
    img = img / 255.0

    
    return img

In [3]:
class L1Dist(Layer):
    
    
    def __init__(self, **kwargs):
        super().__init__()
       
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [4]:
#os.path.join('application_data', 'input_image', 'input_image.jpg')

In [5]:
#os.listdir(os.path.join('application_data', 'verification_images'))

In [6]:
# for image in os.listdir(os.path.join('application_data', 'verification_images')):
#     validation_img = os.path.join('application_data', 'verification_images', image)
#     print(validation_img)

In [7]:
iface_pass_model = tf.keras.models.load_model('iface_passv2.h5', 
                                   custom_objects={'L1Dist': L1Dist, 'BinaryCrossentropy': tf.losses.BinaryCrossentropy})

In [8]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [13]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    frame_height, frame_width, _ = frame.shape
    roi_size = 250

    # Calcul des coordonnées pour centrer la ROI
    roi_y = (frame_height - roi_size) // 2
    roi_x = (frame_width - roi_size) // 2

    # Extraction de la ROI centrée
    frame = frame[roi_y:roi_y + roi_size, roi_x:roi_x + roi_size, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):


        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(iface_pass_model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 126ms/step
True


In [15]:
import os

directory = os.path.join('application_data', 'verification_images')

# Parcourir tous les fichiers dans le répertoire
for file in os.listdir(directory):
    # Vérifier si le fichier est caché (commence par un point)
    if file.startswith('.'):
        file_path = os.path.join(directory, file)
        # Supprimer le fichier
        os.remove(file_path)
        print(f"Fichier caché supprimé : {file}")


In [19]:
np.sum(np.squeeze(results) > 0.9)

50

: 

In [12]:
45/50

0.9